###  Introducción a los modelos de lenguaje en NLP

Los **modelos de lenguaje** son herramientas fundamentales en el campo del Procesamiento del Lenguaje Natural (NLP). Su principal tarea consiste en asignar una probabilidad a secuencias de palabras, ya sea para predecir la siguiente palabra en una oración o para evaluar la probabilidad de que una oración completa sea gramatical y tenga sentido.  
  
Por ejemplo, consideremos la secuencia:

> **Please turn your homework ...**

Un modelo de lenguaje intentará asignar probabilidades a cada palabra que pudiera continuar esta oración. Es muy probable que palabras como "in" o "over" tengan altas probabilidades, mientras que términos poco probables como "refrigerador" o "the" en ese contexto recibirán probabilidades bajas. Esta capacidad predictiva es utilizada en diversas aplicaciones:  

- **Corrección gramatical y ortográfica:** Ayuda a identificar y corregir errores (por ejemplo, distinguir entre "Their" y "There").  
- **Reconocimiento de voz:** Permite que un sistema decida entre alternativas similares en función de la probabilidad de ocurrencia.  
- **Sistemas de comunicación aumentativa:** Facilita la selección de palabras en dispositivos para personas con dificultades para comunicarse.

Los grandes modelos de lenguaje actuales se entrenan precisamente con la tarea de predecir palabras futuras, y con ello aprenden de manera implícita una gran cantidad de información sobre la estructura y el significado del lenguaje.


Los modelos de lenguaje también pueden asignar una probabilidad a una oración completa. Por ejemplo, pueden predecir que la siguiente secuencia tiene una probabilidad mucho mayor de aparecer en un texto:

**`all of a sudden I notice three guys standing on the sidewalk`** que esta misma secuencia de palabras en un orden diferente: **`on guys all I of notice sidewalk three a sudden standing the`**




####  Modelos de lenguaje basados en n-gramas

Una de las aproximaciones clásicas para construir un modelo de lenguaje es la utilización de **n-gramas**. Un n-grama es una secuencia contigua de _n_ palabras extraída de un corpus. Por ejemplo:  
- **Unigrama (n = 1):** Considera palabras individuales.  
- **Bigramas (n = 2):** Considera pares consecutivos de palabras.  
- **Trigramas (n = 3):** Considera secuencias de tres palabras, y así sucesivamente.

La idea central es utilizar la frecuencia con la que aparecen estas secuencias en un corpus para estimar la probabilidad de ocurrencia de una palabra dada una historia (contexto).

Consideremos el ejemplo de predecir la siguiente palabra en la secuencia "its water is so transparent that". La probabilidad condicional de la palabra siguiente (por ejemplo, "the") se puede estimar mediante el cociente:

$$
P(\text{the}|\text{its water is so transparent that}) = \frac{C(\text{its water is so transparent that the})}{C(\text{its water is so transparent that})}
$$

Donde $C(\cdot)$ indica la frecuencia o conteo de aparición de la secuencia en un corpus lo suficientemente grande. Si bien este método de estimación basado en frecuencias relativas es intuitivo, el lenguaje es tan creativo y diverso que incluso un corpus enorme (como toda la web) puede no cubrir todas las secuencias posibles, generando problemas con datos escasos.

#### La regla de la cadena y la descomposición de probabilidades

Si quisiéramos conocer la probabilidad conjunta de una secuencia completa de palabras como *"its water is so transparent"*, podríamos calcularla preguntándonos: *"De todas las posibles secuencias de cinco palabras, ¿cuántas corresponden a 'its water is so transparent'?"* Para ello, deberíamos contar la ocurrencia de *"its water is so transparent"* y dividirla entre la suma de los recuentos de todas las posibles secuencias de cinco palabras.  

Dado que este enfoque es computacionalmente ineficiente, necesitamos introducir métodos más inteligentes para estimar la probabilidad de una palabra `w` dada una historia `h`, o la probabilidad de una secuencia completa de palabras `W`.  

Para formalizar la notación, representaremos la probabilidad de que una variable aleatoria $X_i$ tome el valor *"the"*, es decir, $P(X_i = \text{"the"})$, mediante la simplificación $P(\text{the})$.  

Denotaremos una secuencia de $n$ palabras como $w_1, w_2, \dots, w_n$ o, de manera equivalente, $w_{1:n}$. La expresión $w_{1:n-1}$ representa la secuencia $w_1, w_2, \dots, w_{n-1}$, y también usaremos la notación alternativa $w_{<n}$, que se puede leer como *"todos los elementos de $w$ desde $w_1$ hasta $w_{n-1}$"*.  

Para expresar la probabilidad conjunta de una secuencia completa de palabras, en lugar de escribir:

$$
P(X_1 = w_1, X_2 = w_2, X_3 = w_3, \dots, X_n = w_n)
$$

usaremos la notación más compacta:

$$
P(w_1, w_2, \dots, w_n).
$$
  

Para calcular la probabilidad de una secuencia completa de palabras $P(w_{1:n})$, aplicamos la **regla de la cadena** de la probabilidad, que nos permite descomponer la probabilidad conjunta en una serie de probabilidades condicionales:

$$
P(w_{1:n}) = P(w_1) \cdot P(w_2 | w_1) \cdot P(w_3 | w_{1:2}) \cdots P(w_n | w_{1:n-1}) = \prod_{k=1}^{n} P(w_k | w_{1:k-1}).
$$

Esta descomposición es fundamental, ya que en lugar de estimar directamente la probabilidad conjunta de una secuencia completa (lo cual sería inabordable en la práctica debido a la explosión combinatoria), podemos estimar la probabilidad condicional de cada palabra dado su contexto previo.  

Sin embargo, en la mayoría de los casos, no conocemos una forma exacta de calcular la probabilidad de una palabra dada una larga secuencia de palabras precedentes, $P(w_n | w_{1:n-1})$, lo que nos lleva a explorar métodos aproximados y modelos de lenguaje que faciliten esta tarea.


**Suposición de Markov en los modelos de lenguaje**

En la práctica, calcular $ P(w_n|w_{1:n-1}) $ resulta difícil debido a la inmensidad y complejidad del contexto completo. Aquí es donde entra la **suposición de Markov**. Esta asume que la probabilidad de una palabra depende únicamente de las últimas $n-1$ palabras y no de toda la historia previa.  
 
Por ejemplo, en el **modelo bigrama** (donde $ n = 2 $) se asume:

$$
P(w_n|w_{1:n-1}) \approx P(w_n|w_{n-1})
$$

Es decir, en lugar de considerar toda la secuencia "Walden Pond's water is so transparent that", solo se utiliza la última palabra "that" para predecir "the". De forma similar, se pueden construir modelos trigramas (considerando las dos palabras anteriores) y, en general, modelos n-gramas:

$$
P(w_n|w_{1:n-1}) \approx P(w_n|w_{n-N+1:n-1})
$$

Esta simplificación permite manejar la complejidad del modelo, pero introduce limitaciones, ya que se ignora la información de largo alcance que pudiera ser relevante para predecir la siguiente palabra.

> Para los modelos probabilísticos, normalizar significa dividir por algún conteo total de manera que las probabilidades resultantes estén entre 0 y 1.


#### Estimación de probabilidades mediante MLE

La **estimación por máxima verosimilitud (MLE)** es un método directo para estimar las probabilidades de ocurrencia basándose en los conteos observados en el corpus de entrenamiento. La idea es que la probabilidad de que ocurra una secuencia de palabras se aproxima a la frecuencia con la que aparece dicha secuencia en el corpus.

Por ejemplo, para un bigrama se tiene:

$$
P(w_n|w_{n-1}) = \frac{C(w_{n-1},w_n)}{C(w_{n-1})}
$$

Aquí, $C(w_{n-1},w_n)$ es la cantidad de veces que aparece el par $(w_{n-1},w_n)$ y $C(w_{n-1})$ es la cantidad total de veces que aparece la palabra $w_{n-1}$ en la posición anterior en el corpus. La misma idea se extiende a n-gramas de orden superior:

$$
P(w_n|w_{n-N+1:n-1}) = \frac{C(w_{n-N+1:n-1},w_n)}{C(w_{n-N+1:n-1})}
$$

Este método, aunque intuitivo, enfrenta dos problemas principales:

- **Escasez de datos:** Un n-grama que nunca se haya observado en el corpus recibe una probabilidad cero, lo que puede ser problemático cuando se trabaja con secuencias nuevas.  
- **Sobreestimación de n-gramas frecuentes:** Los n-gramas que aparecen muy a menudo pueden tener sus probabilidades sobreestimadas, sesgando el modelo.



##### **Ejemplo con un mini-corpus y cálculo de probabilidades de bigramas**  

Trabajaremos con un ejemplo utilizando un mini-corpus de tres oraciones. Para ello, primero debemos modificar cada oración agregando un símbolo especial `<s>` al inicio, lo que nos proporcionará el contexto de bigrama para la primera palabra. También añadiremos un símbolo especial de fin de oración `</s>` para indicar el final de la secuencia.  

> El símbolo de fin de oración es necesario para garantizar que la gramática de bigramas defina una distribución de probabilidad válida. Sin este símbolo, en lugar de que la suma de las probabilidades de todas las oraciones sea igual a uno, las probabilidades de las oraciones de una longitud específica sumarían uno. Esto implicaría un conjunto infinito de distribuciones de probabilidad, con una distribución distinta para cada longitud de oración.  

Nuestro mini-corpus con los símbolos `<s>` y `</s>` quedaría de la siguiente forma:  

```
<s> I am Sam </s>  
<s> Sam I am </s>  
<s> I do not like green eggs and ham </s>  
```

A partir de este corpus, calculamos algunas probabilidades de bigramas:

$$
P(\text{I} | \texttt{<s>}) = \frac{2}{3} = 0.67
$$

$$
P(\text{Sam} | \texttt{<s>}) = \frac{1}{3} = 0.33
$$

$$
P(\text{am} | \text{I}) = \frac{2}{3} = 0.67
$$

$$
P(\texttt{</s>} | \text{Sam}) = \frac{1}{2} = 0.5
$$

$$
P(\text{Sam} | \text{am}) = \frac{1}{2} = 0.5
$$

$$
P(\text{do} | \text{I}) = \frac{1}{3} = 0.33
$$


En el caso general de estimación de parámetros para modelos de n-gramas mediante **máxima verosimilitud (MLE)**, la probabilidad condicional se calcula como:

$$
P(w_n | w_{n-N+1:n-1}) = \frac{C(w_{n-N+1:n-1} w_n)}{C(w_{n-N+1:n-1})}
$$

Esta ecuación estima la probabilidad de un n-grama dividiendo la frecuencia observada de la secuencia completa ($w_{n-N+1:n-1}, w_n$) por la frecuencia del prefijo ($w_{n-N+1:n-1}$), es decir, el contexto previo sin la última palabra.  

La idea detrás de esta ecuación es que la probabilidad de que una palabra ocurra en un contexto específico se puede aproximar observando con qué frecuencia aparece dicho contexto completo en el corpus de entrenamiento, en relación con la frecuencia del prefijo. Esto refleja cuán probable es que una palabra específica siga a un conjunto particular de palabras anteriores.  

El uso de frecuencias relativas para estimar probabilidades es un ejemplo de **estimación de máxima verosimilitud (MLE)**. En **MLE**, el conjunto de parámetros encontrados maximiza la verosimilitud del conjunto de entrenamiento $T$ dado el modelo $M$, es decir, $P(T | M)$,


##### **Tokenización y preprocesamiento del corpus**

Antes de construir las tablas de frecuencia, es fundamental procesar el corpus para convertir el texto en una lista de tokens (palabras) de manera que se pueda trabajar con ellos de forma estructurada.

El siguiente fragmento de código muestra una función para tokenizar el texto:

In [ ]:
import re
from collections import defaultdict

def tokenize(texto):
    # Tokeniza el texto y remueve signos de puntuación
    texto = re.sub(r'[^\w\s]', '', texto.lower())
    return texto.split()

# Ejemplo de tokenización
corpus = """
    What restaurants serve Italian food in Berkeley?
    Where can I find a cheap restaurant downtown?
"""
tokens = tokenize(corpus)
print(tokens)


##### **Construcción de tablas de frecuencia**

Una vez tokenizado el texto, se procede a construir tablas de frecuencia para unigrama, bigramas y trigramas. Estas tablas serán esenciales para estimar las probabilidades según MLE.

El código a continuación muestra cómo construir un contador de n-gramas:



In [ ]:
from collections import Counter

def build_ngram_counts(tokens, n):
    ## Se generan n-gramas utilizando la técnica de zip para crear versiones desplazadas de la lista de tokens.
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return Counter(ngrams)

# Conteo de unigramas, bigramas y trigramas
unigrams = build_ngram_counts(tokens, 1)
bigrams = build_ngram_counts(tokens, 2)
trigrams = build_ngram_counts(tokens, 3)

print("Unigramas:", unigrams)
print("Bigramas:", bigrams)
print("Trigramas:", trigrams)


Línea clave: `ngrams = zip(*[tokens[i:] for i in range(n)])`

1. `tokens[i:] for i in range(n)`: Crea una lista de sublistas (o iteradores) que son versiones desplazadas de la lista original de tokens. Por ejemplo, si tenemos los tokens `['a', 'b', 'c', 'd']` y `n=2` (bigramas), esto generará dos versiones:

- `tokens[0:] --> ['a', 'b', 'c', 'd']` (sin desplazamiento)
- `tokens[1:] → ['b', 'c', 'd']` (desplazado en una posición)

2. `zip(*...)`: Combina las sublistas generadas en el paso anterior, de manera que produce tuplas que representan las secuencias de n-gramas. Siguiendo el ejemplo de bigramas, zip tomaría un elemento de cada lista desplazada y los combinaría en tuplas:

- La primera tupla sería `('a', 'b')`
- La segunda tupla sería `('b', 'c')`
- La tercera tupla sería `('c', 'd')`

Para trigramas, el proceso es similar pero con tres listas desplazadas.

3. `Counter(ngrams)`: Después de crear las tuplas de n-gramas con zip, se las pasa a Counter, que cuenta cuántas veces aparece cada tupla (n-grama) en el texto.

##### **Estimación de probabilidades con MLE**

Una vez que se han obtenido las tablas de frecuencia, se pueden calcular las probabilidades de ocurrencia de n-gramas utilizando el método de máxima verosimilitud (MLE).

Para un bigrama, la fórmula MLE es:


$P(w_n | w_{n-1}) = \frac{C(w_{n-1} w_n)}{C(w_{n-1})}$

El código siguiente implementa esta fórmula:

In [ ]:
def bigram_prob(bigrams, unigrams, word1, word2):
    return bigrams[(word1, word2)] / unigrams[(word1,)]

# Ejemplo de probabilidad de bigrama
word1, word2 = 'cheap', 'restaurant'
print(f"P({word2}|{word1}) =", bigram_prob(bigrams, unigrams, word1, word2))

##### **Generalización a n-gramas**

La misma idea se extiende para n-gramas de orden superior. La fórmula general es: 

$P(w_n|w_{n-N+1:n-1}) = \frac{C(w_{n-N+1:n-1} w_n)}{C(w_{n-N+1:n-1)}}$

El siguiente código implementa la función para calcular la probabilidad de un n-grama.

In [ ]:
def ngram_prob(ngram_counts, lower_order_counts, context, word):
    return ngram_counts[context + (word,)] / lower_order_counts[context]

# Ejemplo con trigramas
context = ('i', 'find')
word = 'a'
print(f"P({word}|{' '.join(context)}) =", ngram_prob(trigrams, bigrams, context, word))


##### **Suavizado de Laplace**

Uno de los problemas de la estimación MLE es que si un n-grama nunca se ha visto en el corpus, su probabilidad se asigna como cero. Esto es problemático para cualquier modelo que deba generalizar a secuencias nuevas.  
El **suavizado de Laplace** (o suavizado aditivo) añade 1 al conteo de cada n-grama para evitar probabilidades nulas:


$P(w_n | w_{n-1}) = \frac{C(w_{n-1} w_n) + 1}{C(w_{n-1}) + |V|}$

donde $|V|$ es el tamaño del vocabulario (la cantidad de palabras únicas).

El siguiente fragmento de código implementa esta técnica:

In [ ]:
from collections import defaultdict

# Función para aplicar suavizado de Laplace
def laplace_smoothing(bigrams, unigrams, word1, word2, vocab_size):
    """
    Calcula la probabilidad suavizada de un bigrama usando el suavizado de Laplace.
    
    
    Parámetros:
    - bigrams: un diccionario con conteos de bigramas.
    - unigrams: un diccionario con conteos de unigramas.
    - word1: la primera palabra del bigrama.
    - word2: la segunda palabra del bigrama.
    - vocab_size: el tamaño del vocabulario (número total de palabras únicas).
    
    Retorna:
    - La probabilidad suavizada del bigrama.
    """
    bigram_count = bigrams[(word1, word2)]
    unigram_count = unigrams[(word1,)]
    return (bigram_count + 1) / (unigram_count + vocab_size)

# Ejemplo de uso
unigrams = defaultdict(int)
bigrams = defaultdict(int)

# Ejemplo de corpus tokenizado
tokens = ['i', 'find', 'a', 'cheap', 'restaurant', 'downtown']

# Construcción de conteos de unigramas y bigramas
for i in range(len(tokens) - 1):
    unigrams[(tokens[i],)] += 1
    bigrams[(tokens[i], tokens[i+1])] += 1
# No olvides contar el último unigrama
unigrams[(tokens[-1],)] += 1

# Tamaño del vocabulario
vocab_size = len(set(tokens))

# Calculamos la probabilidad suavizada para el bigrama ('cheap', 'restaurant')
word1, word2 = 'cheap', 'restaurant'
prob = laplace_smoothing(bigrams, unigrams, word1, word2, vocab_size)
print(f"Probabilidad suavizada de ('{word1}', '{word2}'):", prob)


##### **Evaluación del modelo: perplejidad**

Una vez construido el modelo, es importante evaluar su desempeño. Una métrica común es la **perplejidad**, que mide la capacidad del modelo para predecir una secuencia de palabras. La perplejidad se define en términos del logaritmo negativo de las probabilidades y, en esencia, mide la incertidumbre del modelo.

El siguiente código calcula la perplejidad de un conjunto de prueba:

In [ ]:
import math

def calculate_perplexity(test_data, bigrams, unigrams):
    perplexity = 0
    N = len(test_data)
    for i in range(1, N):
        word1, word2 = test_data[i-1], test_data[i]
        prob = laplace_smoothing(bigrams, unigrams, word1, word2, vocab_size)
        perplexity += -math.log(prob)
    return math.exp(perplexity / N)

test_tokens = tokenize("find a restaurant downtown")
print("Perplejidad:", calculate_perplexity(test_tokens, bigrams, unigrams))


##### **Interpolación de n-gramas**

El suavizado y la combinación de distintos órdenes de n-gramas pueden mejorar la estimación de probabilidades. La **interpolación** busca combinar las estimaciones de unigramas, bigramas y trigramas (u órdenes superiores) para obtener una probabilidad final más robusta. La fórmula de interpolación es:

$$
P_{interp}(w_n | w_{n-1}, w_{n-2}) = \lambda_3 P(w_n | w_{n-1}, w_{n-2}) + \lambda_2 P(w_n | w_{n-1}) + \lambda_1 P(w_n)
$$

Donde $\lambda_1$, $\lambda_2$, $\lambda_3$ son los pesos asignados a los unigramas, bigramas y trigramas, respectivamente que suman 1 y determinan la contribución de cada modelo de orden diferente.


In [ ]:
from collections import defaultdict

# Función para obtener la probabilidad interpolada
def interpolated_prob(trigrams, bigrams, unigrams, word1, word2, word3, vocab_size, lambdas=(0.1, 0.3, 0.6)):
    """
    Calcula la probabilidad interpolada usando unigramas, bigramas y trigramas.
    P(w_n | w_{n-2}, w_{n-1}) = λ3 * P(w_n | w_{n-2}, w_{n-1}) + λ2 * P(w_n | w_{n-1}) + λ1 * P(w_n)

    Parámetros:
    - trigrams: diccionario con conteos de trigramas.
    - bigrams: diccionario con conteos de bigramas.
    - unigrams: diccionario con conteos de unigramas.
    - word1: palabra n-2.
    - word2: palabra n-1.
    - word3: palabra n.
    - vocab_size: tamaño del vocabulario.
    - lambdas: pesos para la interpolación.

    Retorna:
    - La probabilidad interpolada.
    """
    # Descomponemos los pesos lambda
    lambda1, lambda2, lambda3 = lambdas

    # Calcular probabilidades individuales (con suavizado Laplace)
    P_w3 = (unigrams[(word3,)] + 1) / (sum(unigrams.values()) + vocab_size)
    P_w3_given_w2 = (bigrams[(word2, word3)] + 1) / (unigrams[(word2,)] + vocab_size)
    P_w3_given_w1_w2 = (trigrams[(word1, word2, word3)] + 1) / (bigrams[(word1, word2)] + vocab_size)

    # Probabilidad interpolada
    return lambda3 * P_w3_given_w1_w2 + lambda2 * P_w3_given_w2 + lambda1 * P_w3

# Ejemplo de datos y uso
tokens = ['i', 'find', 'a', 'cheap', 'restaurant', 'downtown']

# Construimos unigramas, bigramas y trigramas
unigrams = defaultdict(int)
bigrams = defaultdict(int)
trigrams = defaultdict(int)

for i in range(len(tokens) - 2):
    unigrams[(tokens[i],)] += 1
    bigrams[(tokens[i], tokens[i+1])] += 1
    trigrams[(tokens[i], tokens[i+1], tokens[i+2])] += 1
unigrams[(tokens[-2],)] += 1
bigrams[(tokens[-2], tokens[-1])] += 1
unigrams[(tokens[-1],)] += 1

vocab_size = len(set(tokens))

# Calcular probabilidad interpolada para ("cheap", "restaurant", "downtown")
word1, word2, word3 = 'cheap', 'restaurant', 'downtown'
prob_interpolada = interpolated_prob(trigrams, bigrams, unigrams, word1, word2, word3, vocab_size)
print(f"Probabilidad interpolada de ('{word1}', '{word2}', '{word3}'):", prob_interpolada)


#### El suavizado de **Kneser-Ney** 

El suavizado de Laplace es útil, pero en algunos casos se requieren técnicas más sofisticadas. El **suavizado de Kneser-Ney** es una de las técnicas avanzadas que no solo utiliza los conteos absolutos de los n-gramas, sino que también tiene en cuenta la diversidad de contextos en los que aparece una palabra. Esto es especialmente útil para manejar n-gramas raros cuyos componentes individuales puedan aparecer en muchos contextos diferentes.

La fórmula para un bigrama con suavizado de Kneser-Ney es:

$
P_{KN}(w_n | w_{n-1}) = \max(C(w_{n-1} w_n) - d, 0) / C(w_{n-1}) + \lambda(w_{n-1}) P_{KN}(w_n)
$

Donde $d$ es un descuento y $\lambda(w_{n-1})$ ajusta la probabilidad restante para asegurar que las probabilidades sumen `1`.


In [ ]:
def kneser_ney_prob(bigrams, unigrams, word1, word2, discount, vocab_size):
    """
    Calcula la probabilidad suavizada de Kneser-Ney para bigramas.
    
    Parámetros:
    - bigrams: diccionario con conteos de bigramas.
    - unigrams: diccionario con conteos de unigramas.
    - word1: primera palabra del bigrama.
    - word2: segunda palabra del bigrama.
    - discount: el factor de descuento.
    - vocab_size: tamaño del vocabulario.
    
    Retorna:
    - La probabilidad de Kneser-Ney para el bigrama.
    """
    # Contar bigramas y unigramas
    bigram_count = bigrams[(word1, word2)]
    unigram_count = unigrams[(word1,)]

    # Descuento aplicado
    max_term = max(bigram_count - discount, 0) / unigram_count

    # Conteo de bigramas distintos que empiezan con word1 (contexto)
    unique_continuations = len([w for w in unigrams if (word1, w[0]) in bigrams])

    # Probabilidad marginal de la palabra individual (unigrama)
    marginal_prob = (unigrams[(word2,)] + 1) / (sum(unigrams.values()) + vocab_size)

    # Probabilidad suavizada
    return max_term + discount * unique_continuations * marginal_prob / unigram_count

# Parámetros
discount = 0.75  # Un descuento típico en Kneser-Ney

# Calcular la probabilidad suavizada de Kneser-Ney para ('cheap', 'restaurant')
word1, word2 = 'cheap', 'restaurant'
prob_kneser_ney = kneser_ney_prob(bigrams, unigrams, word1, word2, discount, vocab_size)
print(f"Probabilidad de Kneser-Ney para ('{word1}', '{word2}'):", prob_kneser_ney)


#### Ejercicios

**Ejercicio 1: Carga y preprocesamiento del corpus**

**Objetivo:** Cargar y preprocesar un corpus de texto.

- **Instrucciones:**
  1. Carga un archivo de texto que contenga diálogos sobre restaurantes (puede simular el corpus del **Berkeley Restaurant Project**).
  2. Implementa una función que procese el texto eliminando signos de puntuación y convierta todo a minúsculas.
  3. Tokeniza el texto en una lista de palabras.

```python
# Archivo corpus.txt
"""
What restaurants serve Italian food in Berkeley?
Where can I find a cheap restaurant downtown?
"""

def load_and_preprocess_corpus(file_path):
    """
    Carga y preprocesa el corpus desde un archivo de texto.
    - Elimina signos de puntuación y convierte todo a minúsculas.
    - Devuelve una lista de palabras (tokens).
    """
    # Implementar la función de carga y preprocesamiento del corpus
    pass

# Tarea: Implementa y prueba la función usando un archivo de texto.
```

**Ejercicio 2: Cálculo decConteos de unigramas, bigramas y trigramas**

**Objetivo:** Implementar un código para calcular los conteos de unigramas, bigramas y trigramas a partir del corpus tokenizado.

- **Instrucciones:**
  1. Escribe una función para contar los unigramas, bigramas y trigramas.
  2. Devuelve los conteos como diccionarios.

```python
from collections import defaultdict

def calculate_ngram_counts(tokens, n):
    """
    Calcula los conteos de n-gramas a partir de una lista de tokens.
    Retorna un diccionario con los conteos de n-gramas.
    """
    # Implementar el cálculo de n-gramas
    pass

# Tarea: Implementa la función para unigramas, bigramas y trigramas
# Ejemplo: tokens = ['what', 'restaurants', 'serve', 'italian', 'food']
```

**Ejercicio 3: Implementación de MLE (Maximum Likelihood Estimation)**

**Objetivo:** Implementar la estimación de máxima verosimilitud (MLE) para bigramas y trigramas.

- **Instrucciones:**
  1. Implementa una función que calcule la probabilidad de un bigrama o trigram usando MLE.
  2. Usa la fórmula:

  $$
  P(w_n | w_{n-1}) = \frac{C(w_{n-1} w_n)}{C(w_{n-1})}
  $$

```python
def mle_prob(bigrams, unigrams, word1, word2):
    """
    Calcula la probabilidad MLE para un bigrama.
    """
    # Implementar la probabilidad MLE
    pass

# Tarea: Calcula P('italian' | 'serve') usando los bigramas y unigramas de tu corpus.
```

**Ejercicio 4: Aplicación de suavizado de Laplace**

**Objetivo:** Implementar el suavizado de Laplace para bigramas.

- **Instrucciones:**
  1. Modifica la función MLE para incluir el suavizado de Laplace, usando la fórmula:

  $$
  P(w_n | w_{n-1}) = \frac{C(w_{n-1} w_n) + 1}{C(w_{n-1}) + |V|}
  $$

  2. Implementa la función y calcula la probabilidad suavizada para algunos bigramas del corpus.

```python
def laplace_smoothing(bigrams, unigrams, word1, word2, vocab_size):
    """
    Aplica el suavizado de Laplace a un bigrama.
    """
    # Implementar suavizado de Laplace
    pass

# Tarea: Calcula la probabilidad suavizada para varios bigramas como ('cheap', 'restaurant').
```

**Ejercicio 5: Implementación de la interpolación de N-gramas**

**Objetivo:** Implementar la interpolación de unigramas, bigramas y trigramas para mejorar las probabilidades.

- **Instrucciones:**
  1. Usa la fórmula de interpolación:

  $$
  P_{interp}(w_n | w_{n-1}, w_{n-2}) = \lambda_3 P(w_n | w_{n-1}, w_{n-2}) + \lambda_2 P(w_n | w_{n-1}) + \lambda_1 P(w_n)
  $$

  2. Implementa una función que combine unigramas, bigramas y trigramas con pesos $\lambda_1$, $\lambda_2$, $\lambda_3$.

```python
def interpolated_prob(trigrams, bigrams, unigrams, word1, word2, word3, vocab_size, lambdas=(0.1, 0.3, 0.6)):
    """
    Calcula la probabilidad interpolada usando unigramas, bigramas y trigramas.
    """
    # Implementar interpolación
    pass

# Tarea: Prueba la interpolación para secuencias de tres palabras del corpus.
```

**Ejercicio 6: Suavizado de Kneser-Ney**

**Objetivo:** Implementar el suavizado de Kneser-Ney para bigramas.

- **Instrucciones:**
  1. Implementa el suavizado de Kneser-Ney para calcular la probabilidad de un bigrama.
  2. Usa la fórmula:

  $$
  P_{KN}(w_n | w_{n-1}) = \frac{\max(C(w_{n-1} w_n) - d, 0)}{C(w_{n-1})} + \lambda(w_{n-1}) P_{KN}(w_n)
  $$

```python
def kneser_ney_prob(bigrams, unigrams, word1, word2, discount, vocab_size):
    """
    Calcula la probabilidad suavizada de Kneser-Ney para bigramas.
    """
    # Implementar suavizado de Kneser-Ney
    pass

# Tarea: Aplica suavizado de Kneser-Ney para calcular la probabilidad de ('cheap', 'restaurant').
```

**Ejercicio 7: Evaluación del modelo con perplejidad**

**Objetivo:** Implementar una función para evaluar el modelo de n-gramas con **perplejidad**.

- **Instrucciones:**
  1. Implementa una función que calcule la **perplejidad** de un modelo n-grama dado un conjunto de prueba.

  $$
  \text{Perplejidad} = \exp \left( - \frac{1}{N} \sum_{i=1}^N \log P(w_i | w_{i-n+1:i-1}) \right)
  $$

```python
import math

def calculate_perplexity(test_data, ngram_model, unigrams, bigrams, trigrams, vocab_size):
    """
    Calcula la perplejidad de un modelo n-grama dado un conjunto de prueba.
    """
    # Implementar la función de perplejidad
    pass

# Tarea: Calcula la perplejidad del modelo con suavizado de Laplace en un conjunto de prueba.
```

In [ ]:
### Tus respuestas

#### **Algunos problemas prácticos**  

Hasta ahora, hemos descrito modelos de **bigramas** con fines pedagógicos. Sin embargo, en la práctica podemos utilizar **modelos de trigramas**, que se condicionan en las dos palabras anteriores, o incluso **modelos de 4-gramas o 5-gramas** cuando se dispone de suficientes datos de entrenamiento.  

Para estos **n-gramas más largos**, es necesario asumir **contextos adicionales** tanto al inicio como al final de la oración.  

Por ejemplo, para calcular probabilidades de trigramas al comienzo de una oración, utilizamos **dos pseudo-palabras `<s>`** que representan el inicio de la oración. Esto nos permite manejar casos en los que los primeros trigramas no cuentan con suficientes palabras previas.  


##### **Ejemplo: Cálculo de Probabilidades de Trigramas**  

Supongamos que queremos calcular la probabilidad del trigrama **"I am happy"** al comienzo de una oración. Específicamente, buscamos estimar:  

$$
P(\text{happy} | \text{I am})
$$

##### **Paso 1: Preparar la oración con pseudo-palabras**  

Dado que "I" es la primera palabra de la oración, agregamos dos pseudo-palabras `<s>` al principio para calcular el primer trigrama. Así, la oración se modifica de la siguiente manera:  

```
<s> <s> I am happy
```

##### **Paso 2: Calcular la probabilidad del primer trigrama**  

El primer trigrama es `(<s>, <s>, I)`, y su probabilidad se calcula como:  

$$
P(\text{I} | <s> <s>)
$$

Esto representa la probabilidad de que "I" sea la primera palabra tras los dos marcadores de inicio de oración.

##### **Paso 3: Calcular probabilidades sucesivas**  

El siguiente trigrama es `(<s>, I, am)`, cuya probabilidad se calcula como:  

$$
P(\text{am} | <s> I)
$$

Finalmente, para el trigrama **"I am happy"**, calculamos:  

$$
P(\text{happy} | I\ am)
$$

Este enfoque permite extender los modelos de n-gramas a secuencias más largas y mejorar su capacidad predictiva.

##### **Observación**  

Siempre representamos y calculamos las probabilidades en modelos de lenguaje utilizando su **forma logarítmica**, conocidas como **probabilidades logarítmicas**.  

Dado que las probabilidades son, por definición, **menores o iguales a 1**, al multiplicar varias de ellas, el producto se vuelve cada vez más pequeño. Multiplicar un número suficientemente grande de **n-gramas** puede llevar a **desbordamiento numérico** debido a la acumulación de valores extremadamente pequeños.  

Para evitar este problema, en lugar de trabajar con probabilidades crudas, utilizamos **logaritmos**. Al hacerlo, obtenemos valores que son más manejables numéricamente y evitamos el riesgo de desbordamiento.  

Además, la **suma en el espacio logarítmico** es equivalente a la **multiplicación en el espacio lineal**, lo que simplifica los cálculos. Es decir, en lugar de multiplicar probabilidades directamente, **sumamos sus logaritmos**:  

$
p_1 \times p_2 \times p_3 \times p_4 = \exp(\log p_1 + \log p_2 + \log p_3 + \log p_4)
$

En la práctica, usamos `log` para referirnos al **logaritmo natural ($\ln$)** cuando la base no se especifica.

